In [ ]:
import numpy as np
import pandas as pd
import warnings
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
import os
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import TransformedTargetRegressor
from utils import utils_gn, utils_dgrd, utils_models
import importlib
importlib.reload(utils_gn)
importlib.reload(utils_models)
importlib.reload(utils_dgrd)
warnings.filterwarnings("ignore")

# Checking the impact of Cycle number threshold on Model accuracy

Here, we fix the number of signature levels and vary the input cycles.

In [ ]:
# load train raw data
train_raw = utils_gn.read_data('train_1238.pkl')

In [ ]:
# Load test data
test_raw = utils_gn.read_data('test_1238.pkl')
y_test_raw = utils_gn.read_data('true_test_labels_1238.pkl')

In [ ]:
list_of_cycles = np.arange(10, 101, 2)
list_of_cycles

### Cross-validation on training set 

In [ ]:
def choosing_best_n_cross_val(target_list, model_type, n_list, fixed_level, ylabels):

        '''
        Function that investigates the effect of cycle number threshold on modelling through cross-validation.
        
        Args:
        ----
                target_list: list of targets to predict
                model_type:  "cycle" (predict cycles) or "capacity-ir" (predict values at cycles)
                n_list:      a list of cycle number thresholds
                fixed_level: a fixed number of signature level
                ylabels:     a list of labels for y-axis

        Returns:
        -------
                dictionary of error metrics.            
        '''
        
        # Split targets based on capacity/IR
        if model_type == "cycle":
                split_list = [target_list[:3], target_list[3:]]
        elif model_type == "capacity-ir":
                split_list = [target_list[:2], target_list[2:]]

        _, ax = plt.subplots(1, 2, figsize=(16, 4))

        dict_of_res = {}

        for i, split in enumerate(split_list):

            this_mae, this_rmse = [], []
            mae_ci, rmse_ci = [], []
        
            for n in n_list:
                print('n: ', n)
                
                # Get training set
                tr = utils_gn.FeatureTransformation(n=n)
                X_train, y_train = tr.fit_transform(data=train_raw, targets=target_list, with_eol=True, sig_level=fixed_level)


                # Build model
                if model_type == "cycle":

                        # Choose the best hyperparameters based on eol
                        model_gs = TransformedTargetRegressor(XGBRegressor(), func=np.log10, inverse_func=utils_models.antilog)
                        params = {'regressor__model__n_estimators': [50, 60, 100],
                                'regressor__model__learning_rate': [0.1, 0.2],
                                'regressor__model__max_depth':[2, 3, 4, 5]
                                }
                        gs = GridSearchCV(estimator=model_gs, param_grid=params, scoring='neg_mean_absolute_error', cv=3).fit(X_train, y_train[:, -1])

                        model = TransformedTargetRegressor(MultiOutputRegressor(XGBRegressor(n_estimators=gs.best_params_['regressor__model__n_estimators'],
                                                                                                max_depth=gs.best_params_['regressor__model__max_depth'],
                                                                                                learning_rate=gs.best_params_['regressor__model__learning_rate'])),
                                                                                func=np.log10,
                                                                                inverse_func=utils_models.antilog)
                elif model_type == "capacity-ir":
                        
                        # Choose the best hyperparameters based on ir at eol
                        model_gs = TransformedTargetRegressor(XGBRegressor(), func=np.log10, inverse_func=utils_models.antilog)
                        params = {'regressor__model__n_estimators': [300, 400, 500],
                                'regressor__model__learning_rate': [0.1, 0.2],
                                'regressor__model__max_depth':[3, 4, 5, 6]
                                }
                        gs = GridSearchCV(estimator=model_gs, param_grid=params, scoring='neg_mean_absolute_error', cv=3).fit(X_train, y_train[:, -1])

                        model = TransformedTargetRegressor(MultiOutputRegressor(XGBRegressor(n_estimators=gs.best_params_['regressor__model__n_estimators'],
                                                                                                max_depth=gs.best_params_['regressor__model__max_depth'],
                                                                                                learning_rate=gs.best_params_['regressor__model__learning_rate'])),
                                                                                func=np.log10,
                                                                                inverse_func=utils_models.antilog)
                # Call repeated k-fold cross-validation on the training set
                val_scores, val_scores_raw = utils_models.kfold_cross_validation(X=X_train, y=y_train, model=model, cv=5)
                
                # Append the scores to the list of metrics
                this_mae.append(val_scores['test_MAE'])
                this_rmse.append(val_scores['test_RMSE'])

                # Calculate the CI
                mae_ci.append(utils_models.confidence_interval_any(values=val_scores_raw['test_MAE'], n_bootstraps=1000, alpha=0.1))
                rmse_ci.append(utils_models.confidence_interval_any(values=val_scores_raw['test_RMSE'], n_bootstraps=1000, alpha=0.1))

            # Cast to numpy array
            mae_ci = np.array(mae_ci)
            rmse_ci = np.array(rmse_ci)
            
            # Store the current results in a dictionary
            dict_of_res['data_'+str(i)] = this_mae, mae_ci, this_rmse, rmse_ci

            ax[i].plot(n_list, this_mae, 'D--', label='MAE', color='blue', markersize=5)
            ax[i].fill_between(n_list, mae_ci[:, 0], mae_ci[:, 1], color='blue', alpha=0.1, label='MAE: 90% CI')

            ax[i].set_xlabel('Input cycle number', fontsize=16)
           
            ax[i].set_ylabel(ylabels[i], fontsize=16)
            ax[i].set_title(', '.join(split), fontsize=18)

            ax[i].plot(n_list, this_rmse, 's-', color='crimson', label='RMSE', markersize=5)
            ax[i].fill_between(n_list, rmse_ci[:, 0], rmse_ci[:, 1], color='crimson', alpha=0.1, label='RMSE: 90% CI')
    
        
        handles, labels = ax[0].get_legend_handles_labels()
        ax[0].legend(handles, labels, loc='upper center', ncol=4, fontsize=16, bbox_to_anchor=(1.0, -0.15))


        if model_type == "cycle":
               plt.savefig(fname="plots/sig_level2_crossval_n_effect_cycle_tabs12.pdf", bbox_inches='tight')
        else:
               plt.savefig(fname="plots/sig_level2_crossval_n_effect_capir_tabs12.pdf", bbox_inches='tight')
        
        return dict_of_res

In [ ]:
# For the model that predicts cycles
dict_cycle_model = choosing_best_n_cross_val(target_list=['k-o', 'k-p', 'EOL', 'e-o', 'e-p'], model_type="cycle", n_list=list_of_cycles, fixed_level=2,
                          ylabels=['Cross-validated errors (cycles)', 'Cross-validated errors (cycles)'])

In [ ]:
# For the model that predicts capacities/internal resistance
dict_capacity_ir_model = choosing_best_n_cross_val(target_list=['Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL'],
                                                   model_type="capacity-ir",
                                                   n_list=list_of_cycles,
                                                   fixed_level=2,
                                                   ylabels=['Cross-validated errors (Ah)', r'Cross-validated errors ($\Omega$)']
)

In [ ]:
#with open(os.path.join("data", "sig_vary_n_cap_ir_model.pkl"), "wb") as fp:
 #   pickle.dump(dict_capacity_ir_model, fp)

### Effect of input cycles on test errors

In [ ]:
def effect_of_cycle_number_test(target_list, model_type, n_list, ylabels, fixed_level=2):

        """
        Function to plot average test errors for different cycle number threshold.

        Args:
        ----
                target_list: list of target to predict
                model_type:  "cycle" (predict cycles) or "capacity-ir" (predict values at cycles)
                n_list:      a list of cycle number thresholds
                ylabels:     a list of y labels
                fixed_level: fixed level of signature

        Returns:
        -------
                a plot of test average errors vs cycle numbers.              
        """
        
        # Split targets based on capacity/IR
        if model_type == "cycle":
                split_list = [target_list[:3], target_list[3:]]
        elif model_type == "capacity-ir":
                split_list = [target_list[:2], target_list[2:]]

        _, ax = plt.subplots(1, 2, figsize=(16, 4))

        for i, split in enumerate(split_list):

                this_mae, this_rmse = [], []
                mae_ci, rmse_ci = [], []
        
                for n in n_list:
                        print('n: ', n)

                        # Get training set
                        tr = utils_gn.FeatureTransformation(n=n)
                        X_train, y_train = tr.fit_transform(data=train_raw, targets=target_list, with_eol=True, sig_level=fixed_level)
                        X_test, y_test = tr.transform(test_raw, sig_level=fixed_level), y_test_raw[target_list].values


                        # Build model
                        if model_type == "cycle":
                                params = {'n_estimators': 100, 'reg_alpha': 0.1, 'max_depth': 2, 'min_samples_split': 3}   # model from Jupyter notebook 5
                                model = TransformedTargetRegressor(MultiOutputRegressor(XGBRegressor(**params)),
                                                                                        func=np.log10,
                                                                                        inverse_func=utils_models.antilog)
                        elif model_type == "capacity-ir":
                                params = {'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.1}    # model from Jupyter notebook 6
                                model = TransformedTargetRegressor(MultiOutputRegressor(XGBRegressor(**params)), 
                                                                                        func=np.log10,
                                                                                        inverse_func=utils_models.antilog)
                        model = model.fit(X_train, y_train)
                        
                        # Get validation scores
                        test_scores = utils_models.metrics_calculator(y_test, model.predict(X_test))
                        this_mae.append(test_scores['MAE'])
                        this_rmse.append(test_scores['RMSE'])

                        error = np.ravel((y_test - model.predict(X_test)))

                        # Calculate the CI
                        mae_ci.append(utils_models.confidence_interval_any(values=abs(error), n_bootstraps=1000, alpha=0.1))
                        rmse_ci.append(utils_models.confidence_interval_any(values=error, n_bootstraps=1000, metric_type='rmse', alpha=0.1))

                
                # Cast to numpy array
                mae_ci = np.array(mae_ci)
                rmse_ci = np.array(rmse_ci)

                ax[i].plot(n_list, this_mae, 'D--', label='MAE', color='blue', markersize=5)
                ax[i].fill_between(n_list, mae_ci[:, 0], mae_ci[:, 1], color='blue', alpha=0.1, label='MAE: 90% CI')

              
                ax[i].set_xlabel('Input cycle number', fontsize=16)
                        
                ax[i].set_ylabel(ylabels[i], fontsize=16)
                ax[i].set_title(', '.join(split), fontsize=18)
               

                ax[i].plot(n_list, this_rmse, 's-', color='crimson', label='RMSE', markersize=5)
                ax[i].fill_between(n_list, rmse_ci[:, 0], rmse_ci[:, 1], color='crimson', alpha=0.1, label='RMSE: 90% CI')
           
        handles, labels = ax[0].get_legend_handles_labels()
        ax[0].legend(handles, labels, loc='upper center', ncol=4, fontsize=16, bbox_to_anchor=(1.0, -0.15))

        if model_type == "cycle":
                plt.savefig(fname="plots/sig_level2_test_n_effect_cycles_tabs12.pdf", bbox_inches='tight')
        else:
                plt.savefig(fname="plots/sig_level2_test_n_effect_capir_tabs12.pdf", bbox_inches='tight')
        



In [ ]:
effect_of_cycle_number_test(target_list=['k-o', 'k-p', 'EOL', 'e-o', 'e-p'],
                            model_type="cycle",
                            n_list=list_of_cycles,
                            ylabels=['Average test errors (cycles)', 'Average test errors (cycles)'],
                            fixed_level=2
)

In [ ]:
effect_of_cycle_number_test(target_list=['Qatk-o', 'Qatk-p', 'IRate-o', 'IRate-p', 'IRatEOL'],
                            model_type="capacity-ir",
                            n_list=list_of_cycles,
                            ylabels=['Average test errors (Ah)', r'Average test errors ($\Omega$)'],
                            fixed_level=2
)